In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re
import langchain
import time
import bertopic
import umap
import hdbscan
import sklearn
import nltk
import requests
from bs4 import BeautifulSoup
from langchain_text_splitters import RecursiveCharacterTextSplitter
from SpeechProcessing import documentImporter   
from Coalitions import scrape_german_coalitions
from SpeechProcessing import dategetter


In [2]:
Abortion_corpus_df = pd.read_csv("/home/pc/Uni/MasterThesis/Scripts/Corpus_Chunked_full_annotated_cleaned.csv", header = 0)
Abortion_corpus_df = Abortion_corpus_df[Abortion_corpus_df['qwen2.5:7b'].notna() & (Abortion_corpus_df['qwen2.5:7b'] == 1)]

In [3]:
# Store patterns in dictionaries for better organization
patterns = {
    "preamble": re.compile(r'("id":"\d{4}")(.*?)Uhr.{0,150}(Alterspräsidentin|Alterspräsident|Vizepräsidentin|Vizepräsident|Vizekanzlerin|Vizekanzler|Präsidentin|Präsident|Kanzlerin|Kanzler).{0,50}?:', re.DOTALL), #I swear this makes sense!
    "appendix": re.compile(r'(\(Schluß der Sitzung: \d+(.|:)\d+ Uhr.?\)|\\nAnlagen zum Stenographischen Bericht|\\nAnlage 1)(.*?)("id":"\d{4})', re.DOTALL),
    "appendix_last": re.compile(r'(\(Schluß der Sitzung: \d+(.|:)\d+ Uhr.?\)|\\nAnlagen zum Stenographischen Bericht|\\nAnlage 1)(.*?)', re.DOTALL),    # Last appendix without "id" at the end
    "party_speaker": re.compile(r'[^\s,]+ [^\s,]+ \([^\s,]+\)\s?:', re.DOTALL),                                                                         # Generic pattern for speeches, e.g. 'Speaker (Party) :'                                           
    "party_speaker_CDU": re.compile(r'[^\s,]+ [^\s,]+ \(CDU/CSU\)\s?:', re.DOTALL),                                                                     # Specific pattern for CDU speeches
    "party_speaker_FDP_random": re.compile(r'[^\s,]+ [^\s,]+ \(F.D.P.\)\s?:', re.DOTALL),                                                               # For an ungodly reason the FDP was briefly referred to as F.D.P. 1999-2000. I suspect this is a conspiracy to sabotage my thesis and social science in general.
    "party_speaker_new": re.compile(r'\[\w+\]\s?', re.DOTALL),                                                                                          # New pattern for speeches after 2013. The previous pattern 'Speaker (Party) :' was replaced with 'Speaker [Party] :' in the Bundestag protocol.
    "party_speaker_CDU_new": re.compile(r'\[CDU+/CSU\]\s?:', re.DOTALL),                                                                                # Specific pattern for CDU speeches
    "minister_speaker": re.compile(r'(?:[^\n,]+,\s+Bundesminister(?:in)?\s+(?:der|für|des)\s+[^\n:]+:)', re.DOTALL | re.UNICODE),                      # Ministers are usually addressed with 'Bundesminister der ... i.e. Finanzen'
    "chancellor_speaker": re.compile(r', (?:(?:Bundes|Vize)?[Kk]anzlerin?):', re.DOTALL),                                                                 # Chancellor speeches are usually addressed with 'Bundeskanzlerin:' or 'Vizekanzlerin:'
    "reactions": re.compile(r'\(\w\w+ (.*?)\)', re.DOTALL),                                                                                             # Reactions are usually in the form '(Applaus)', '(Beifall)', '(Zuruf)', these simple reactions are removed here
    "remarks": re.compile(r'\((?!CDU/CSU|CDU|CSU|SPD|FDP|F.D.P.|AfD|Die Linke|Bündnis 90/Die Grünen|Bündnis 90 / Die Grüne|Die Grünen|LINKE|PDS|Piraten|NPD|REP|DVU|ÖDP|Tierschutzpartei|MLPD|DKP|BP|SSW|Fraktionslos)[^(]*?:[^()]+\)', re.DOTALL) # Excludes party markers i.e. --> Joachim Gauck (CDU) : I need to keep these to identify individual speeches which
}

keywords = [
    "Schwangerschaftsabbruch",
    "Abtreibung",
    "abgetrieben",
    "Abtreibungsgesetz",
    "Abtreibungsrecht",
    "§ 218",
    "Schwangerschaft abgebrochen",
    "Schwangerschaft",
    "Schwanger",
    "Paragraf 218",
    "Schwangerschaftskonfliktgesetz",
    "Fristenlösung",
    "Indikationsregelung",
    "Beratungspflicht",
    "Beratungsschein",
    "werdendes Leben",
    "ungeborenes Kind",
    "ungeborenes Leben",
    "werdende Mutter",
    "Strafbarkeit Schwangerschaftsabbruch",
    "Entkriminalisierung",
    "Legalisierung",
    "Abbruchsversorgung",
    "medizinische Indikation",
    "kriminologische Indikation",
    "embryopathische Indikation",
    "Schwangerschaftskonfliktberatung",
    "Ärzt*innen Schwangerschaftsabbruch",
    "Arzt Schwangerschaftsabbruch",
    "Kostenübernahme Abtreibung",
    "Gesundheitsversorgung Schwangere",
    "Komplikationen Schwangerschaftsabbruch",
    "psychologische Betreuung",
    "Selbstbestimmungsrecht",
    "reproduktive Rechte",
    "reproduktive Selbstbestimmung",
    "Frauenrechte",
    "Stigmatisierung Abbruch",
    "Tabu Abtreibung",
    "Lebensschutz",
    "ungewollt schwanger",
    "Indikationslösung",
    "Fristenlösung",
    "Schutz des ungeborenen Lebens",
    "Diskriminierung Schwangerer",
    "Versorgung ungewollt Schwangerer",
    "Bundestagsdebatte Schwangerschaftsabbruch",
    "Parlamentsdebatte Abtreibung",
    "Gesetzesentwurf Schwangerschaftsabbruch",
    "Gesetzesänderung §218",
    "Antrag Schwangerschaftsabbruch",
    "Abstimmung Schwangerschaftsabbruch",
    "Expertenkommission Schwangerschaftsabbruch",
    "Öffentlichkeit Schwangerschaftsabbruch",
    "Gruppenantrag Abtreibung",
    "Ampel-Koalition Abtreibung",
    "CDU/CSU Position Abtreibung",
    "Liberalisierung Abtreibungsrecht",
    "Werbeverbot Schwangerschaftsabbruch",
    "Paragraf 219a",
    "Beratungsregel",
    "Kompromiss Schwangerschaftsabbruch",
    "Verfassungsgericht Urteil Schwangerschaftsabbruch",
    "Spätabbruch",
    "Minderjährige Schwangere",
    "Schwangere Jugendliche",
    "religiöse Verbände Abtreibung",
    "Statistik Schwangerschaftsabbruch",
    "Wir haben abgetrieben",
    "219a Werbeverbot",
    "Schwangerschaftsabbruch EU",
    "Internationaler Vergleich Abtreibungsrecht",
    "ungewollt schwanger",
    "ungeborenes Leben",
    "Aufhebung Werbeverbot",
    "Pro Familia",
    "SPD Schwangerschaftsabbruch",
    "Grüne Schwangerschaftsabbruch",
    "FDP Schwangerschaftsabbruch",
    "Linke Schwangerschaftsabbruch",
    "AfD Schwangerschaftsabbruch",
    "Abtreibungskonflikt",
    "Kindstötung", 
    "Memmingen"
]

In [4]:
text_splitter_pre2013 = RecursiveCharacterTextSplitter(
    
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=True,

    separators=[
        patterns['party_speaker'].pattern,
        patterns['party_speaker_CDU'].pattern,
        patterns['chancellor_speaker'].pattern,
        patterns['minister_speaker'].pattern,
        patterns['party_speaker_FDP_random'].pattern, # Strange pattern that occurs in the parliamentary protocols around 1999 idk why. 
        r'[.!?]+\s+', 
        r'\n\n',     
        r'\n',       
        r'\s+',      
    ]
)

# The utilization of two splitter is necessary due to some structural changes in the protocols after 2013 as the patterns changed. Don't as me  why, I just work here.

text_splitter_post2013 = RecursiveCharacterTextSplitter(

    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=True,
    keep_separator=True,
    
    separators=[
    patterns['chancellor_speaker'].pattern,
    patterns['minister_speaker'].pattern,
    patterns['party_speaker_CDU'].pattern,  
    patterns['party_speaker'].pattern,
    r'[.!?]+\s+', 
    r'\n\n',       
    r'\n',         
    r'\s+',       
    ]
)


In [5]:
import nltk
nltk.download('punkt')

def split_into_sentences(text):
    """Split text into individual sentences using NLTK"""
    sentences = nltk.sent_tokenize(text, language='german')
    # Filter out very short sentences (likely fragments)
    return [sent.strip() for sent in sentences if len(sent.strip()) > 20]

# Modified splitting function
Abortion_corpus_df_split = []

for idx, row in Abortion_corpus_df.iterrows():
    date_str = str(row['date'])
    year_match = re.search(r'datetime\.date\((\d{4})', date_str)
    
    if year_match:
        year = int(year_match.group(1))
        
        # Use NLTK sentence tokenizer for precise sentence splitting
        sentences = split_into_sentences(row['chunk'])
        
        # Create new rows for each sentence
        for sentence in sentences:
            new_row = row.copy()
            new_row['chunk'] = sentence
            new_row['Year'] = year
            Abortion_corpus_df_split.append(new_row)

# Convert list to DataFrame
Abortion_corpus_df_split = pd.DataFrame(Abortion_corpus_df_split)
print(f"Original chunks: {len(Abortion_corpus_df)}")
print(f"Split sentences: {len(Abortion_corpus_df_split)}")
print(f"Average sentence length: {Abortion_corpus_df_split['chunk'].str.len().mean():.1f} characters")

[nltk_data] Downloading package punkt to /home/pc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Original chunks: 2245
Split sentences: 67286
Average sentence length: 137.8 characters


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from HanTa import HanoverTagger as ht
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('german'))

enhanced_parliamentary_stopwords = [
    # Basic parliamentary titles and roles
    "herr", "frau", "dr", "prof", "präsident", "präsidentin", "vizepräsident", "vizepräsidentin","männer", "frauen", "herren" , "damen"
    "abgeordneter", "abgeordnete", "abgeordneten", "kollege", "kollegin", "kollegen", "kolleginnen",
    "staatssekretär", "staatssekretärin", "bundesminister", "ministerin", "minister",
    
    # Parliamentary procedure terms
    "sitzung", "beratung", "ausschuss", "fraktion", "drucksache", "gesetzentwurf", "entwurf",
    "zusatzfrage", "zwischenfrage", "wort", "damen", "herren", "meine", "verehrten",
    "parl", "abs", "reform", "beschluss", "antrag", "vorlage",
    
    # Party abbreviations and names
    "afd", "spd", "cdu", "csu", "fdp", "grüne", "linke", "bündnis", "fraktion",
    "cdu csu", "cdu/csu", "csu fraktion", "spd fraktion",
    
    # Common parliamentary phrases
    "damen herren", "verehrten damen", "liebe", "vielen dank", "dank", "bitte",
    "ärztinnen ärzte", "kolleginnen kollegen", "meine damen", "sehr geehrte",
    
    # Time references and procedural terms
    "heute", "jahr", "donnerstag", "bonn", "berlin", "bundestag", "bundesrat",
    "sagen", "müssen", "gibt", "schon", "ja", "mehr", "mal", "wissen", "möchte",
    "treffen", "entscheiden", "stimmt", "woche", "seitdem", "entwicklung",
    
    # Generic procedural words
    "000", "dm", "millionen", "ausgaben", "auftreten", "bemerkung", "zugleich",
    "zweite", "größte", "aussehen", "spricht", "alltag", "wortlaut",
    
    # Specific names that appear frequently (from your results)
    "hans", "peter", "klein", "thorsten", "lieb", "axel", "müller", "marco", "buschmann",
    "kroll", "schlüter", "christoph", "kerstin", "engelhard", "funcke", "scheu", "schleicher",
    "schäfer", "andre", "detlef", "parr", "waigel", "winkelmeier", "becker", "ostman",
    "pfeifer", "neumeister", "bärbel", "bas", "katrin", "helling", "göring", "eckardt",
    "plahr", "wolfgang", "kubicki", "jäger", "burkhard", "kinkel", "hirsch", "wurbs",
    "schoeler",
    
    # Generic transition words and fillers
    "geht", "muß", "unserer", "gesagt", "sei", "auch", "schleicher", "warum",
    "hier", "diese", "dann", "noch", "über", "nach", "durch", "aber", "nicht",
    "eine", "einer", "einem", "eines", "dass", "sich", "werden", "wurde", "wird",
    "haben", "hatte", "sind", "war", "waren", "sowie", "bzw", "etc", "usw",
    
    # Parliamentary location references
    "bonn", "berlin", "deutschen", "bundestag", "bundesrat", "deutschland", "ländern",
    
    # Numbers and dates
    "2008", "02", "15", "1979", "1978", "1975", "218", "219a", "stgb",
    
    # Common verbs in parliamentary context
    "sagen", "müssen", "wollen", "können", "sollen", "mögen", "lassen", "machen",
    "gehen", "kommen", "stehen", "liegen", "setzen", "nehmen", "geben", "bringen"
]

sklearn_stopwords_de = list(stopwords.words('german'))

stopwords = sklearn_stopwords_de + enhanced_parliamentary_stopwords

def stop_word_removal(x):
    token = x.lower().split()  # Convert to lowercase first
    return ' '.join([w for w in token if not w in stopwords])

def preprocess(text):
    # Remove parliamentary titles and names
    text = re.sub(r'\b(herr|frau|dr|prof|abgeordneter?)\s+\w+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(vizepräsident|präsident|staatssekretär)\w*\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(beratung|drucksache|sitzung|ausschuss)\b', '', text, flags=re.IGNORECASE)
    
    # Clean text
    text = re.sub(r'[^\w\s]', ' ', text)  
    text = re.sub(r'\d+', ' ', text)    
    text = re.sub(r'\s+', ' ', text)      
    text = text.strip()                   
    text = re.sub(r'\b\w\b', '', text)  
    
    # Process tokens
    tokens = text.lower().split()
    meaningful_tokens = []
    
    for token in tokens:
        if (len(token) > 2 and 
            token not in enhanced_parliamentary_stopwords and 
            token not in sklearn_stopwords_de and
            not token.isdigit()):
            token = [lemma for (word,lemma,pos) in tagger.tag_sent(token.split())][0].lower() 
            meaningful_tokens.append(token)
    
    return ' '.join(meaningful_tokens)

Abortion_corpus_df_split['preprocessed'] = Abortion_corpus_df_split['chunk'].apply(stop_word_removal)


[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
abortion_keywords = [
    ["volkskörper", "bevölkerungswachstum", "volksgesundheit"],
    ["schutzlos", "hilflos", "schutzbedürftig", "schutzbedürftige", "schutzbedürftigen", "schutzbedürftigkeit"],
    ["eugenik", "kranke kinder", "erbschäden", "erbgut", "behinderung", "behindert", "beeinträchtigt", "erbkrank"],
    ["werdendes leben", "ungeborenes kind", "ungeborenes leben", "lebensschutz", "schutz werdenden lebens", "recht auf leben"],
    ["selbstbestimmungsrecht", "reproduktive selbstbestimmung", "frauenrechte", "patriarchat", "patriarchal", "patriarchalisch", "menschenwürde", "würde der frau"],
    ["medizinische indikation", "kriminologische indikation", "embryopathische indikation"],
    ["hilfsbedürftig", "entscheidungshilfe", "mädchen", "beratung","beratungsbedarf", "in not geraten", "notlagen", "frauen in not"],
    ["individuell", "selbstbestimmung", "psychologisch", "psychisch", "psychische gesundheit"],
    ["pfuscher", "kriminelle", "skrupellos", "ausnutzen", "abzocke", "männer", "partner", "vater"],
    ["liberalisierung", "abschaffung", "feminismus", "frauenbewegung", "frauen", "recht auf abtreibung", "patriarchat", "würde"],
    ["soziale gerechtigkeit", "ungerecht", "zugang", "klassen", "ungleich", "tourismus"],
    ["aufgabe", "staat", "staatlicher auftrag", "rolle", "aufgabe", "signal", "zeigen", "klarstellen"],
    ["pragmatismus", "pragmatisch", "praktisch", "lösungsorientiert", "zielorientiert", "funktioniert nicht", "nicht reduziert", "reduziert"],
    ["unfähig", "ungeeignet", "schlechte eltern", "vernachlässigung", "asozial"],
    ["zeugung", "aufklärung", "kompetenz", "förderung", "sexualerziehung", "verhütung", "verhindern"],
    ["soziales problem", "gesellschaftliche folgen", "gesellschaftliche auswirkungen", "gesellschaftliche konsequenzen", "gesellschaft"],
    ["osten", "ddr", "ostdeutsch", "ostdeutsche", "ostdeutschen", "ostfrau", "frauen im osten", "frauen in der ddr", "einigungsvertrag"],
    ["memmingen", "stern", "reportage", "ard", "zdf", "skandal"], 
    ["urteil", "rechtsprechung", "bundesverfassungsgericht", "bverfg", "karlsruhe", "verfassungsgericht", "richter", "rechtsprechung", "1975", "urteile", "verfassung", "grundgesetz"],
    ["christ", "christlich", "evangelisch", "katholisch", "bischof", "kirchen"], 
    ["schwangerschaftskonflikt", "konflikt", "rechtsgut", "rechtsgüter"],
    ["soziale hilfen", "rahmen", "kindergarten", "kinderbetreuung", "erziehungsgeld", "erziehungsurlaub"]
]

In [8]:
theoretical_keywords = [
    ["bundesverfassungsgericht", "bverfg", "karlsruhe", "verfassungsgericht", "richter", "rechtsprechung", "urteile", "verfassung", "grundgesetz", "artikel 2", "urteil", "richter", "rechtsprechung"],
    ["eigenverantwortlich", "selbstbestimmt", "eigenverantwortung", "selbstbestimmung"],
    ["pflicht", "lebensschutz", "schutzpflicht", "staatliche schutzpflicht", "signalwirkung", "schutzbedürftig"],
    ["recht auf leben", "lebensrecht"]
]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sentence_transformers import SentenceTransformer

from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
import torch

embedding_model = SentenceTransformer("all-mpnet-base-v2", device="cuda")  
documents = Abortion_corpus_df_split[(Abortion_corpus_df_split['Year'] < 2026) & (Abortion_corpus_df_split['Year'] > 2010)]['preprocessed'].tolist()

torch.cuda.empty_cache() 
embeddings = embedding_model.encode(documents, batch_size=16)


In [19]:
import openai
import hdbscan
from bertopic import BERTopic
from bertopic.representation import OpenAI
from bertopic.dimensionality import BaseDimensionalityReduction
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

client = openai.OpenAI(
    base_url = 'http://localhost:11434/v1', #wherever ollama is running
    api_key='ollama', # required, but unused
)

representation_model = {
    "mistral:7b": OpenAI(client, model='mistral:7b')
}

empty_dimensionality_model = BaseDimensionalityReduction()

cluster_model2 = KMeans(n_clusters=25)

cluster_model = AgglomerativeClustering(n_clusters=50)

hdbscan_model = HDBSCAN(
    min_cluster_size=10,  
    min_samples=3, 
    metric='euclidean', 
    cluster_selection_method='eom',
    prediction_data=True
)

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)


topic_model = BERTopic(
    language="german",
    nr_topics="auto",
    umap_model=empty_dimensionality_model,
    hdbscan_model=cluster_model2,
    embedding_model= embedding_model,
    representation_model=representation_model,
    seed_topic_list=abortion_keywords,
    ctfidf_model=ctfidf_model,
    calculate_probabilities=True)

from nltk.corpus import stopwords as nltk_stopwords
STOPWORDS = nltk_stopwords.words('german')

vectorizer_model_de = CountVectorizer(stop_words=STOPWORDS)

topics = topic_model.fit_transform(documents, embeddings)
topic_model.update_topics(documents, vectorizer_model=vectorizer_model_de)

In [20]:
res = topic_model.get_topic_info()
print(topic_model.get_topic_info())

    Topic  Count                                               Name  \
0       0   1575                    0_wirklich_ganz_debatte_einfach   
1       1   1345         1_entscheidung_sagen_gesetzentwurf_debatte   
2       2    997                            2_tun_deshalb_gut_dafür   
3       3    911                   3_frauen_frau_frage_frauenrechte   
4       4    638  4_deutschland_bundesregierung_bundestag_deutschen   
5       5    608             5_fraktion_abgeordneten_drucksache_afd   
6       6    559                     6_politik_union_demokratie_spd   
7       7    463                       7_jahr_jahren_euro_millionen   
8       8    423  8_schwangerschaftsabbruch_schwangerschaftsabbr...   
9       9    334  9_gesellschaft_gesellschaftlichen_gesellschaft...   
10     10    324               10_leben_ungeborenen_lebens_menschen   
11     11    317           11_20_abgeordneten_weiterer_abgeordneter   
12     12    316                                  12_dr_csu_cdu_spd   
13    

In [21]:
topic_model.visualize_barchart()
topic_model.visualize_topics()

In [22]:
topic_model.visualize_hierarchy(custom_labels=True)

In [ ]:
# First, convert Year to numeric and create timestamps
years = pd.to_numeric(Abortion_corpus_df_split['Year'], errors='coerce')
# Create timestamps from years (using January 1st of each year)
timestamps = pd.to_datetime(years, format='%Y', errors='coerce')

# Calculate topics over time
topics_over_time = topic_model.topics_over_time(documents, timestamps)

# Fix topic names that might be lists instead of strings
topics_over_time['Name'] = topics_over_time['Name'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))

# Visualize topics over time using relative frequencies (use the modified dataframe)
topic_model.visualize_topics_over_time(topics_over_time)

In [16]:
import openai
import hdbscan
from bertopic import BERTopic
from bertopic.representation import OpenAI
from bertopic.dimensionality import BaseDimensionalityReduction
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

client = openai.OpenAI(
    base_url = 'http://localhost:11434/v1', #wherever ollama is running
    api_key='ollama', # required, but unused
)

representation_model = {
    "mistral:7b": OpenAI(client, model='mistral:7b')
}

empty_dimensionality_model = BaseDimensionalityReduction()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Create German vectorizer model
vectorizer_model_de = CountVectorizer(
    ngram_range=(1, 2),
    stop_words=list(stop_words),
    min_df=2,
    max_df=0.95
)



def create_labeled_bertopic_timeranges(df, time_ranges, topic_keywords, embedding_model):
    """
    Create BERTopic models for different time ranges with labeled topics
    
    Parameters:
    df: DataFrame with preprocessed text and Year column
    time_ranges: List of tuples (start_year, end_year, label)
    topic_keywords: List of keyword lists for seeded topics
    embedding_model: Pre-trained sentence transformer model
    
    Returns:
    Dictionary with time range labels as keys and BERTopic models as values
    """
    
    # Define time ranges for analysis
    time_ranges = [
        (1950, 1980, "Phase-1"),
        (1981, 1990, "Phase-2"),
        (1991, 2025, "Phase-3")
    ]
    
    bertopic_models = {}
    
    for start_year, end_year, label in time_ranges:
        print(f"Processing {label} ({start_year}-{end_year})...")
        
        # Filter data for current time range
        period_df = df[(df['Year'] >= start_year) & (df['Year'] <= end_year)]
        
        if len(period_df) < 50:  # Skip if too few documents
            print(f"Skipping {label}: only {len(period_df)} documents")
            continue
            
        documents_period = period_df['preprocessed'].tolist()
        
        # Generate embeddings for this period
        embeddings_period = embedding_model.encode(documents_period, batch_size=16)
        
        cluster_model_period = HDBSCAN(
                min_cluster_size=max(10, len(documents_period)//100),
                min_samples=5,  # Increase from 3 to reduce sensitivity
                metric='euclidean',
                cluster_selection_method='eom',
                prediction_data=False,  # Set to False - you don't need soft clustering
                core_dist_n_jobs=-1,  # Use all CPU cores
                algorithm='best'  # Let HDBSCAN choose the fastest algorithm
            )
        
        topic_model_period = BERTopic(
            language="german",
            nr_topics="auto",
            umap_model=empty_dimensionality_model,
            hdbscan_model=cluster_model_period,
            embedding_model=embedding_model,
            representation_model=representation_model,
            seed_topic_list=abortion_keywords,
            ctfidf_model=ctfidf_model,
            verbose=True
        )
        
        # Fit the model
        topics_period = topic_model_period.fit_transform(documents_period, embeddings_period)
        topic_model_period.update_topics(documents_period, vectorizer_model=vectorizer_model_de)
        
        # Store model and metadata
        bertopic_models[label] = {
            'model': topic_model_period,
            'topics': topics_period,
            'documents': documents_period,
            'embeddings': embeddings_period,
            'period_df': period_df,
            'years': (start_year, end_year),
            'doc_count': len(documents_period)
        }
        
        print(f"Completed {label}: {len(documents_period)} documents, {len(topic_model_period.get_topic_info())} topics")
    
    return bertopic_models

# Execute the modular time range analysis
bertopic_timeranges = create_labeled_bertopic_timeranges(
    Abortion_corpus_df_split, 
    None,  # Will use default ranges defined in function
    theoretical_keywords, 
    embedding_model
)

# Display summary of results
print("\n=== BERTopic Time Range Analysis Summary ===")
for period_label, model_data in bertopic_timeranges.items():
    topic_info = model_data['model'].get_topic_info()
    print(f"{period_label}: {model_data['doc_count']} docs, {len(topic_info)} topics")

Processing Phase-1 (1950-1980)...


2025-10-22 14:48:07,660 - BERTopic - Guided - Find embeddings highly related to seeded topics.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-22 14:48:07,824 - BERTopic - Guided - Completed ✓
2025-10-22 14:48:07,825 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-22 14:48:07,826 - BERTopic - Dimensionality - Completed ✓
2025-10-22 14:48:07,877 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-22 14:50:40,007 - BERTopic - Cluster - Completed ✓
2025-10-22 14:50:40,008 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-10-22 14:50:40,132 - BERTopic - Representation - Completed ✓
2025-10-22 14:50:40,132 - BERTopic - Topic reduction - Reducing number of topics
2025-10-22 14:50:40,141 - BERTopic - Representation - Fine-tuning topics using representation models.
100%|██████████| 4/4 [00:01<00:00,  2.02it/s]
2025-10-22 14:50:42,258 - BERTopic - Representation - Completed ✓
2025-10-22 14:50:42,260 - BERTopic - Topic reduction - Reduced number of topics from 4 to 4


Completed Phase-1: 11453 documents, 4 topics
Processing Phase-2 (1981-1990)...


...

Completed Phase-2: 10449 documents, 3 topics
Processing Phase-3 (1991-2025)...


2025-10-22 14:53:30,045 - BERTopic - Guided - Find embeddings highly related to seeded topics.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-22 14:53:30,428 - BERTopic - Guided - Completed ✓
2025-10-22 14:53:30,429 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-22 14:53:30,433 - BERTopic - Dimensionality - Completed ✓
2025-10-22 14:53:30,765 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-22 15:32:27,889 - BERTopic - Cluster - Completed ✓
2025-10-22 15:32:27,889 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-10-22 15:32:28,290 - BERTopic - Representation - Completed ✓
2025-10-22 15:32:28,291 - BERTopic - Topic reduction - Reducing number of topics
2025-10-22 15:32:28,302 - BERTopic - Representation - Fine-tuning topics using representation models.
100%|██████████| 3/3 [00:05<00:00,  1.98s/it]
2025-10-22 15:32:34,630 - BERTopic - Representation - Completed ✓
2025-10-22 15:32:34,634 - BERTopic - Topic reduction - Reduced number of topics from 3 to 3


Completed Phase-3: 45384 documents, 3 topics

=== BERTopic Time Range Analysis Summary ===
Phase-1: 11453 docs, 4 topics
Phase-2: 10449 docs, 3 topics
Phase-3: 45384 docs, 3 topics


In [37]:
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))["data"]

In [30]:
from bertopic import BERTopic
year_mask = (Abortion_corpus_df_split['Year'] < 2026) & (Abortion_corpus_df_split['Year'] > 2011)
filtered_df = Abortion_corpus_df_split.loc[year_mask]
Docs = filtered_df['preprocessed'].tolist()

In [31]:
topic_model = BERTopic(verbose=True).fit(Docs)

2025-10-25 14:09:57,817 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/358 [00:00<?, ?it/s]

2025-10-25 14:10:01,522 - BERTopic - Embedding - Completed ✓
2025-10-25 14:10:01,523 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-25 14:10:03,936 - BERTopic - Dimensionality - Completed ✓
2025-10-25 14:10:03,937 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-25 14:10:04,386 - BERTopic - Cluster - Completed ✓
2025-10-25 14:10:04,390 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-25 14:10:04,607 - BERTopic - Representation - Completed ✓


In [32]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3595,-1_ist_ganz_haben_immer,"[ist, ganz, haben, immer, deshalb, werden, knn...","[darf arzt durchgeführt werden, abtreibung vor..."
1,0,658,0_schwangerschaft_schwangerschaftsabbruch_schw...,"[schwangerschaft, schwangerschaftsabbruch, sch...","[zunächst: schwangere recht haben, entscheiden..."
2,1,264,1_bundesregierung_bundesverfassungsgericht_bun...,"[bundesregierung, bundesverfassungsgericht, bu...","[entwickelt sich, sagt bundesverfassungsgerich..."
3,2,188,2_gesellschaft_gesellschaftlichen_gesellschaft...,"[gesellschaft, gesellschaftlichen, gesellschaf...","[kinder zukunft gesellschaft., gesellschaft, l..."
4,3,184,3_wirklich_natrlich_endlich_nmlich,"[wirklich, natrlich, endlich, nmlich, tatschli...",[schluss partyvideos walzerklängen steht grins...
...,...,...,...,...,...
183,182,11,182_afdfraktion_vertreten_humbug_afdverband,"[afdfraktion, vertreten, humbug, afdverband, r...","[wenig afd-fraktion vertreten., afd-verband wa..."
184,183,11,183_parl_franke_edgar_franziska,"[parl, franke, edgar, franziska, giffey, flach...","[dr. edgar franke, parl., dr. edgar franke, pa..."
185,184,10,184_schofel_schaffen_rechtssicherheit_unterste...,"[schofel, schaffen, rechtssicherheit, unterste...","[betroffenen rechtssicherheit schaffen, ärztin..."
186,185,10,185_geld_gelst_fahren_hotel,"[geld, gelst, fahren, hotel, lehrttigkeit, ent...","[wer geld straftaten verdient, volle härte rec..."
